In [36]:
#spark home setup in CMD before running the below code. 

#setx SPARK_HOME C:\Users\VV232LY\spark3   

import findspark
findspark.init()

In [48]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType,FloatType,StringType
from pyspark.sql.functions import col,round
#from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

In [49]:
import json
retail_schemas = json.load(open('C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db_schema.json'))


In [4]:
columns = []
for i in retail_schemas['orders']:
    #print(i['column_name'])
    columns.append(i['column_name'])
    
print(columns)

['order_id', 'order_date', 'order_customer_id', 'order_status']


In [ ]:
#retail_schemas['orders']
columns = list(map(lambda col: col['column_name'], retail_schemas['orders']))
print(columns)

In [5]:
import pandas as pd
input = pd.read_csv('C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db/orders/part-00000', names=columns)

In [7]:
orders = spark.createDataFrame(input)


In [15]:
import re
df=orders.toDF(*[re.sub('[^A-Za-z0-9_]','',col) for col in orders.columns])

df.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [ ]:
#Get Daily Product Revenue 
1. One to Many relationship b/w orders and order_items

In [5]:
columns_order_items = ""
schema = ""
for i in retail_schemas['order_items']:
    schema = schema +" "+ i['column_name'] +" "+ i['data_type'] + ","
    
columns_order_items = schema[:-1]
print(columns_order_items)

 order_item_id int, order_item_order_id int, order_item_product_id int, order_item_quantity int, order_item_subtotal float, order_item_product_price float


In [50]:
order_item_schema = StructType(
                    [StructField("order_item_id", IntegerType(), True) ,
                     StructField("order_item_order_id", IntegerType(), True) ,
                     StructField("order_item_product_id", IntegerType(), True) ,
                     StructField("order_item_quantity", IntegerType(), True) ,
                     StructField("order_item_subtotal", FloatType(), True) ,
                     StructField("order_item_product_price", FloatType(), True) 
                    ]
                    )

#order_item_schema ='order_item_id int, order_item_order_id int,order_item_product_id int, order_item_quantity int, order_item_subtotal float,order_item_product_price float'

In [51]:
order_items= spark.read \
            .format('csv') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db/order_items/part-00000.csv') \
            .schema(order_item_schema) \
            .load()


In [107]:
orders_json= spark.read \
            .format('json') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db_json/orders/part-r-00000.json') \
            .load()


In [108]:
orders_json.show()

+-----------------+--------------------+--------+---------------+
|order_customer_id|          order_date|order_id|   order_status|
+-----------------+--------------------+--------+---------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|
|             7130|2013-07-25 00:00:...|       6|       COMPLETE|
|             4530|2013-07-25 00:00:...|       7|       COMPLETE|
|             2911|2013-07-25 00:00:...|       8|     PROCESSING|
|             5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|             5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
|              918|2013-07-25 00:00:...|      11| PAYMENT_REVIEW|
|             1837|2013-07-25 00:00:...|      12|         CLOSED|
|         

In [ ]:
###Spark Read from mysql:

df=spark.read.format("jdbc") \
             .option("driver","com.mysql.cj.jdbc.Driver") \
             .option("user",username) \
             .option("password",password) \
             .option("url","jdbc:mysql://Mysql.dbmstutorials.com:3306") \
             .option("dbtable","tutorial_db.department") \
             .load()

##Read with Custom query: 

df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",
                                     user="tutorial_user",
                                     password="user_password",
                                     url="jdbc:mysql://Mysql.dbmstutorials.com:3306?serverTimezone=UTC&useSSL=false",
                                     query="SELECT * FROM tutorial_db.department WHERE dept_no IN (100,200)"
                                     ).load()

In [113]:
product_schema = StructType(
                    [StructField("product_id", IntegerType(), True) ,
                     StructField("product_category_id", IntegerType(), True) ,
                     StructField("product_name", StringType(), True) ,
                     StructField("product_description", StringType(), True) ,
                     StructField("product_price", FloatType(), True) ,
                     StructField("product_image", StringType(), True) 
                    ]
                    )


#order_item_schema ='order_item_id int, order_item_order_id int,order_item_product_id int, order_item_quantity int, order_item_subtotal float,order_item_product_price float'

In [115]:
columns_products = ""
schema = ""
for i in retail_schemas['products']:
    schema = schema +" "+ i['column_name'] +" "+ i['data_type'] + ","
    
columns_products = schema[:-1]
print(columns_products)

 product_id int, product_category_id int, product_name string, product_description string, product_price float, product_image string


In [116]:
product= spark.read \
            .format('csv') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db/products/part-00000.csv') \
            .schema(columns_products) \
            .load()

product.show()


+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|       199.99|http://images.acm...|
|         6|                  2|Jordan Men's VI R...|               null|       134.99|http://images.acm...|
|         7|       

In [117]:
product_json= spark.read \
            .format('json') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db_json/products/part-r-00000') \
            .load()

product_json.show()


+-------------------+-------------------+----------+--------------------+--------------------+-------------+
|product_category_id|product_description|product_id|       product_image|        product_name|product_price|
+-------------------+-------------------+----------+--------------------+--------------------+-------------+
|                  2|                   |         1|http://images.acm...|Quest Q64 10 FT. ...|        59.98|
|                  2|                   |         2|http://images.acm...|Under Armour Men'...|       129.99|
|                  2|                   |         3|http://images.acm...|Under Armour Men'...|        89.99|
|                  2|                   |         4|http://images.acm...|Under Armour Men'...|        89.99|
|                  2|                   |         5|http://images.acm...|Riddell Youth Rev...|       199.99|
|                  2|                   |         6|http://images.acm...|Jordan Men's VI R...|       134.99|
|                  

In [120]:
product_json.createOrReplaceTempView("products")

In [144]:

query = """
select product_name,avg(product_price) as avg_price from products
group by product_name
"""
#print(type(query))
output = spark.sql(query).withColumn('avg_price1',round(col('avg_price'),2))

output.show()

+--------------------+------------------+----------+
|        product_name|         avg_price|avg_price1|
+--------------------+------------------+----------+
|Quest Q64 10 FT. ...|59.980000000000004|     59.98|
|Under Armour Men'...|129.98999999999995|    129.99|
|Under Armour Men'...|             89.99|     89.99|
|Riddell Youth Rev...|            199.99|    199.99|
|Jordan Men's VI R...|            134.99|    134.99|
|Schutt Youth Recr...|             99.99|     99.99|
|Nike Men's Vapor ...|            129.99|    129.99|
|Nike Adult Vapor ...|              50.0|      50.0|
|Fitness Gear 300 ...|            209.99|    209.99|
|Under Armour Men'...|            139.99|    139.99|
|Quik Shade Summit...|            199.99|    199.99|
|Under Armour Kids...|             59.99|     59.99|
|Riddell Youth 360...|            299.99|    299.99|
|Reebok Men's Full...|             29.97|     29.97|
|Nike Men's Finger...|124.98999999999998|    124.99|
|Under Armour Kids...|             54.99|     

In [ ]:
1.Get number of order by date or Status
2.Get Revenue by each order id. 
3. Get Daily product revenue. 

In [6]:
order_items.printSchema()

root
 |-- order_item_id: integer (nullable = true)
 |-- order_item_order_id: integer (nullable = true)
 |-- order_item_product_id: integer (nullable = true)
 |-- order_item_quantity: integer (nullable = true)
 |-- order_item_subtotal: float (nullable = true)
 |-- order_item_product_price: float (nullable = true)



In [22]:
from pyspark.sql import functions as F
subtotal = order_items.groupBy('order_item_order_id') \
                      .agg(F.round(F.sum('order_item_subtotal'),2).alias('revenue'),
                                   F.round(F.avg('order_item_subtotal'),2).alias('avg_revenue')
                          ) \
                     .orderBy(col('avg_revenue').desc(),col('revenue').desc())
                    
subtotal.show()
print(subtotal)

+-------------------+-------+-----------+
|order_item_order_id|revenue|avg_revenue|
+-------------------+-------+-----------+
|              68875|2399.95|    1199.97|
|              68837|2299.96|    1149.98|
|              68883|2149.99|    1074.99|
|              68848|2399.96|     799.99|
|              68859|2349.89|      783.3|
|              68736|2259.95|     753.32|
|              68722|2199.99|     733.33|
|              68822|1399.95|     699.97|
|              68809|2779.86|     694.97|
|              68703|3449.91|     689.98|
|              68806|2629.92|     657.48|
|              68821|2629.92|     657.48|
|              68723|1239.95|     619.97|
|              68811| 599.99|     599.99|
|              68724|2859.89|     571.98|
|              68858|2839.91|     567.98|
|              68766| 2699.9|     539.98|
|              68778| 2629.9|     525.98|
|              61904|  500.0|      500.0|
|              68748| 999.97|     499.99|
+-------------------+-------+-----

In [83]:
#task 1: Total Number of Records: 
#order_items.show()
#print(order_items.count())

from pyspark.sql import functions as F

grouped_order_item= order_items.groupBy(F.col('order_item_order_id')) \
                                .agg(F.round(F.sum(F.col('order_item_subtotal')),2).alias('total_amount_per_order'),
                                    F.round(F.avg(F.col('order_item_subtotal')),2).alias('avg_amount_per_order'),
                                     F.max(F.col('order_item_quantity')).alias('max_qty'),
                                     F.min(F.col('order_item_quantity')).alias('min_qty'),
                                    ) \
                                .orderBy(F.col('total_amount_per_order').desc(),F.col('avg_amount_per_order')) \
                                .withColumn('profit',F.expr("case when avg_amount_per_order >= 200 then 'Y' else 'N' end")) \
                                .groupBy(F.col('profit')).agg(F.count(F.col('profit')).alias('cnd_profit'))
                            

    
#grouped_order_item.grou
# print(type(grouped_order_item))
 
#grouped_order_item.show()


#new_df = grouped_order_item.


# new_order_item_df = order_items.filter(F.col('order_item_order_id').isNotNull())

# print(new_order_item_df.count())  300
# print(order_items.count())




+------+----------+
|profit|cnd_profit|
+------+----------+
|     Y|     25283|
|     N|     32148|
+------+----------+



In [11]:
order_items.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

In [19]:
#Task 2: Avg order amount per order.
from pyspark.sql import functions as F

avg_order_amt = order_items.groupBy(col('order_item_id')).agg(F.round(F.avg(F.col('order_item_subtotal')),2).alias('avg_amt_per_order'))

avg_order_amt.show()


+-------------+-----------------+
|order_item_id|avg_amt_per_order|
+-------------+-----------------+
|          148|           399.96|
|          463|           399.98|
|          471|            150.0|
|          496|            59.99|
|          833|           399.98|
|         1088|           299.95|
|         1238|            59.99|
|         1342|           239.96|
|         1580|           199.99|
|         1591|           199.99|
|         1645|           399.98|
|         1829|           399.98|
|         1959|           129.99|
|         2122|            59.97|
|         2142|            79.98|
|         2366|           199.98|
|         2659|           299.98|
|         2866|             50.0|
|         3175|           129.99|
|         3749|           399.98|
+-------------+-----------------+
only showing top 20 rows



In [86]:
#Task 2: Max and Min order order_item_quantity .

min_max_order_amt = order_items.groupBy(col('order_item_order_id')) \
                           .agg(F.round(F.min(F.col('order_item_subtotal')),2).alias('min_amt_per_order'),
                                F.round(F.max(F.col('order_item_subtotal')),2).alias('max_amt_per_order'),
                                F.count(F.col('order_item_subtotal')).alias('number_of_items'),
                                F.round(F.sum(F.col('order_item_subtotal'))).alias('total_orders_value')
                               ) \
                            .filter(F.col('number_of_items') > 1) \
                            .orderBy(F.col('number_of_items').desc(),
                                     F.col('max_amt_per_order').desc())

min_max_order_amt.show()

+-------------------+-----------------+-----------------+---------------+------------------+
|order_item_order_id|min_amt_per_order|max_amt_per_order|number_of_items|total_orders_value|
+-------------------+-----------------+-----------------+---------------+------------------+
|              68724|            59.99|          1999.99|              5|            2860.0|
|              68778|           129.99|          1999.99|              5|            2630.0|
|              68766|            49.98|          1999.99|              5|            2700.0|
|              68858|            100.0|          1999.99|              5|            2840.0|
|              68703|           199.95|          1999.99|              5|            3450.0|
|              68780|            99.96|           999.99|              5|            1730.0|
|              68786|            59.99|           999.99|              5|            1590.0|
|              68733|            59.99|           599.99|             

In [87]:
#Task 3: Window Aggregate: Duplicate removal

from pyspark.sql import Window

running_total_window = Window.partitionBy('order_item_order_id') \
                            .orderBy('total_orders_value') \
                            .rowsBetween(Window.unboundedPreceding,Window.currentRow)


new_df = min_max_order_amt.withColumn('rank',F.row_number().over(running_total_window))

new_df.show()




Py4JJavaError: An error occurred while calling o1315.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 171.0 failed 1 times, most recent failure: Lost task 0.0 in stage 171.0 (TID 158) (IN3017677W1.ey.net executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTas